# Self-Supervised Monocular Depth Estimation

In [15]:
from dataloader import KittiDataset
from resnet import MyNet

from torch.utils.data import DataLoader

import os
import torchvision.transforms as transforms

## data loading

In [16]:
train_path = os.path.join(os.getcwd(), "data/train")
# test_path = os.path.join(os.getcwd(), 'data/test')

In [17]:
train_transform = transforms.Compose([transforms.ToTensor()])
test_transform = transforms.Compose([transforms.ToTensor()])

In [18]:
train_set = KittiDataset(train_path, 'train', transform = train_transform)
# test_set = KittiLoader(test_path, 'test', transform = test_transform)

train_loader = DataLoader(train_set, batch_size = 5, shuffle = True)

for batch_step, batch_sample in enumerate(train_loader):
    print(batch_step, batch_sample)
    break
    

0 {'left_image': tensor([[[[0.5333, 0.5333, 0.5294,  ..., 0.1216, 0.1255, 0.1294],
          [0.5255, 0.5176, 0.4980,  ..., 0.1529, 0.1294, 0.0902],
          [0.6745, 0.6627, 0.6431,  ..., 0.1490, 0.1137, 0.0549],
          ...,
          [0.3529, 0.3647, 0.3490,  ..., 0.4118, 0.7451, 0.9451],
          [0.3686, 0.3686, 0.3490,  ..., 0.3804, 0.7255, 1.0000],
          [0.3725, 0.3725, 0.3608,  ..., 0.7176, 0.7059, 0.7725]],

         [[0.4196, 0.4078, 0.3961,  ..., 0.1569, 0.1451, 0.1294],
          [0.5137, 0.4902, 0.4667,  ..., 0.1569, 0.1373, 0.1176],
          [0.5922, 0.5725, 0.5490,  ..., 0.1608, 0.1294, 0.0902],
          ...,
          [0.3608, 0.3569, 0.3490,  ..., 0.6863, 0.8863, 0.8784],
          [0.3451, 0.3333, 0.3373,  ..., 0.8667, 0.9961, 0.9412],
          [0.3294, 0.3216, 0.3255,  ..., 0.8353, 1.0000, 0.9843]],

         [[0.2941, 0.2471, 0.2353,  ..., 0.1137, 0.1255, 0.1216],
          [0.4706, 0.4941, 0.4863,  ..., 0.1176, 0.1294, 0.1294],
          [0.4745, 0.5255

## Implement CNN using PyTorch

In [19]:
net = MyNet().eval()